# Lab 6.1 — Summarizing the Healthcare Survey with PySpark

**Files included in `/content/data`:**
- `health_survey_v2.csv` — survey responses
- `ReverseCodingItems_v2.csv` — reverse-coding metadata
- `health_survey_summary.csv` — final produced summary (also produced by this notebook if you run it)

Follow the instructions and run cells sequentially. Screenshots placeholders are provided where your instructor requests them.


In [2]:
!apt-get install -y openjdk-11-jdk-headless -qq > /dev/null
!pip install -q pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Lab6_1").getOrCreate()

import os



In [8]:
!mkdir -p data

!curl -o data/health_survey_v2.csv \
https://raw.githubusercontent.com/theisenr05/Lab_6.1/refs/heads/main/data/health_survey_v2.csv

!curl -o data/ReverseCodingItems_v2.csv \
https://raw.githubusercontent.com/theisenr05/Lab_6.1/refs/heads/main/data/ReverseCodingItems_v2.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  195k  100  195k    0     0   740k      0 --:--:-- --:--:-- --:--:--  742k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3157  100  3157    0     0  13068      0 --:--:-- --:--:-- --:--:-- 13099


In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, when, create_map, lit, regexp_extract

spark = SparkSession.builder.master('local[*]').appName('Lab6_1').getOrCreate()

survey = spark.read.csv('/content/data/health_survey_v2.csv', header=True, inferSchema=True)
reverse = spark.read.csv('/content/data/ReverseCodingItems_v2.csv', header=True, inferSchema=True)

survey.show(5, truncate=False)
reverse.show(5, truncate=False)


+---+--------------+--------------------------+--------------------------+--------------+--------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------+--------------------------+--------------------------+-----------------+--------------------------+--------------+-----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------------------+--------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------+-----------------+--------------+-----------------+--------------------------+--------------+--------------------------+--------------------------+--------------------------+--------------+--------------------------+--------------+
|ID |F1            |F5                        |F2         

In [14]:
question_cols = [c for c in survey.columns if c != 'ID']
print('Question columns (count={}):'.format(len(question_cols)), question_cols)

stack_expr = "stack({}, {})".format(len(question_cols), ", ".join([f"'{c}', `{c}`" for c in question_cols]))
long_df = survey.select(col('ID'), expr(stack_expr).alias('Question','Response'))
long_df.show(20, truncate=False)


Question columns (count=44): ['F1', 'F5', 'F2', 'F1_1', 'F2_1', 'F6', 'F4', 'F3', 'F5_1', 'F1_2', 'F2_2', 'F6_1', 'F2_3', 'F4_1', 'F2_4', 'F5_2', 'F2_5', 'F6_2', 'F1_3', 'F2_6', 'F5_3', 'F4_2', 'F2_7', 'F3_1', 'F2_8', 'F5_4', 'F3_2', 'F1_4', 'F3_3', 'F1_5', 'F5_5', 'F6_3', 'F1_6', 'F5_6', 'F2_9', 'F3_4', 'F4_3', 'F2_10', 'F1_7', 'F6_4', 'F4_4', 'F5_7', 'F3_5', 'F2_11']
+---+--------+-----------------+
|ID |Question|Response         |
+---+--------+-----------------+
|1  |F1      |Somewhat Agree   |
|1  |F5      |Somewhat Disagree|
|1  |F2      |Somewhat Agree   |
|1  |F1_1    |Somewhat Agree   |
|1  |F2_1    |Somewhat Agree   |
|1  |F6      |Somewhat Disagree|
|1  |F4      |Somewhat Agree   |
|1  |F3      |Somewhat Agree   |
|1  |F5_1    |Somewhat Agree   |
|1  |F1_2    |Somewhat Agree   |
|1  |F2_2    |Somewhat Agree   |
|1  |F6_1    |Somewhat Agree   |
|1  |F2_3    |Somewhat Agree   |
|1  |F4_1    |Somewhat Agree   |
|1  |F2_4    |Somewhat Agree   |
|1  |F5_2    |Somewhat Agree   |
|

In [16]:
reverse_small = reverse.select(col('Column Name').alias('Question'), col('Needs Reverse Coding?').alias('NeedsReverse'))
joined = long_df.join(reverse_small, on='Question', how='left').fillna({'NeedsReverse':'No'})

coded = joined.withColumn('Temp_Normal',
    when(col('Response')=='Strongly Disagree', 1)
    .when(col('Response')=='Somewhat Disagree', 2)
    .when(col('Response')=='Neither Agree nor Disagree', 3)
    .when(col('Response')=='Somewhat Agree', 4)
    .when(col('Response')=='Strongly Agree', 5)
    .otherwise(None)
).withColumn('Temp_Reverse',
    when(col('Response')=='Strongly Disagree', 5)
    .when(col('Response')=='Somewhat Disagree', 4)
    .when(col('Response')=='Neither Agree nor Disagree', 3)
    .when(col('Response')=='Somewhat Agree', 2)
    .when(col('Response')=='Strongly Agree', 1)
    .otherwise(None)
)

coded = coded.withColumn('RecodedValue', when(col('NeedsReverse')=='Yes', col('Temp_Reverse')).otherwise(col('Temp_Normal')))

coded = coded.withColumn('QuestionType', regexp_extract(col('Question'), r'^(F\d+)', 0))

coded.show(20, truncate=False)


+--------+---+-----------------+------------+-----------+------------+------------+------------+
|Question|ID |Response         |NeedsReverse|Temp_Normal|Temp_Reverse|RecodedValue|QuestionType|
+--------+---+-----------------+------------+-----------+------------+------------+------------+
|F1      |1  |Somewhat Agree   |No          |4          |2           |4           |F1          |
|F5      |1  |Somewhat Disagree|Yes         |2          |4           |4           |F5          |
|F2      |1  |Somewhat Agree   |No          |4          |2           |4           |F2          |
|F1_1    |1  |Somewhat Agree   |No          |4          |2           |4           |F1          |
|F2_1    |1  |Somewhat Agree   |No          |4          |2           |4           |F2          |
|F6      |1  |Somewhat Disagree|Yes         |2          |4           |4           |F6          |
|F4      |1  |Somewhat Agree   |No          |4          |2           |4           |F4          |
|F3      |1  |Somewhat Agree  

In [20]:
from pyspark.sql.functions import sum as spark_sum

agg_long = (
    coded
    .groupBy("ID", "QuestionType")
    .agg(spark_sum("RecodedValue").alias("RecodedTotal"))
    .orderBy("ID", "QuestionType")
)

agg_long.show(50, truncate=False)

+---+------------+------------+
|ID |QuestionType|RecodedTotal|
+---+------------+------------+
|1  |F1          |31          |
|1  |F2          |48          |
|1  |F3          |20          |
|1  |F4          |17          |
|1  |F5          |28          |
|1  |F6          |18          |
|2  |F1          |31          |
|2  |F2          |47          |
|2  |F3          |19          |
|2  |F4          |17          |
|2  |F5          |27          |
|2  |F6          |20          |
|3  |F1          |36          |
|3  |F2          |46          |
|3  |F3          |19          |
|3  |F4          |18          |
|3  |F5          |32          |
|3  |F6          |17          |
|4  |F1          |32          |
|4  |F2          |54          |
|4  |F3          |12          |
|4  |F4          |15          |
|4  |F5          |30          |
|4  |F6          |16          |
|5  |F1          |37          |
|5  |F2          |47          |
|5  |F3          |22          |
|5  |F4          |19          |
|5  |F5 

In [21]:
wide_df = (
    agg_long
    .groupBy("ID")
    .pivot("QuestionType")
    .agg(spark_sum("RecodedTotal"))
    .orderBy("ID")
)

wide_df.show(truncate=False)

+---+---+---+---+---+---+---+
|ID |F1 |F2 |F3 |F4 |F5 |F6 |
+---+---+---+---+---+---+---+
|1  |31 |48 |20 |17 |28 |18 |
|2  |31 |47 |19 |17 |27 |20 |
|3  |36 |46 |19 |18 |32 |17 |
|4  |32 |54 |12 |15 |30 |16 |
|5  |37 |47 |22 |19 |36 |19 |
|6  |36 |51 |20 |23 |37 |24 |
|7  |36 |56 |22 |19 |29 |22 |
|8  |39 |49 |18 |21 |29 |16 |
|9  |36 |52 |21 |22 |30 |19 |
|10 |35 |58 |28 |19 |40 |23 |
|11 |33 |48 |20 |19 |31 |20 |
|12 |38 |56 |28 |22 |29 |19 |
|13 |30 |45 |18 |16 |33 |18 |
|14 |30 |52 |23 |18 |31 |18 |
|15 |30 |49 |17 |19 |30 |23 |
|16 |30 |54 |26 |23 |31 |11 |
|17 |32 |55 |18 |18 |29 |21 |
|18 |33 |51 |6  |14 |22 |23 |
|19 |36 |55 |21 |19 |39 |21 |
|20 |31 |41 |17 |16 |27 |16 |
+---+---+---+---+---+---+---+
only showing top 20 rows



In [22]:
wide_df.write.csv("data/health_survey_summary.csv", header=True, mode="overwrite")